In [ ]:
import os

directory = '/Skin-Cancer-Detection-App/backend/detection/data/labels'

if not os.path.exists(directory):
    os.makedirs(directory)
    print(f"Directory created: {directory}")
else:
    print(f"Directory already exists: {directory}")

Directory created: /Users/zcengiz/Grad-Project/Skin-Cancer-Detection-App/data/labels


In [ ]:
import numpy as np
import cv2
import os
import pandas as pd

def mask_to_bounding_box(image_path):

    df = pd.read_csv("/Skin-Cancer-Detection-App/backend/detection/data/metadata.csv")  
    
    img_name = os.path.basename(image_path).split('.')[0]

    if img_name not in df['image'].values:
        print(f"Warning: {img_name} not found in metadata.csv. Skipping...")
        return  

    mask = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    _, binary_mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    bounding_boxes = []

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)

        img = cv2.imread(image_path)  
        img_height, img_width = img.shape[:2]

        x_norm = x / img_width
        y_norm = y / img_height
        w_norm = w / img_width
        h_norm = h / img_height

        mapping = {'MEL': 0, 'NV': 1, 'BCC': 2, 'AKIEC': 3, 'BKL': 4, 'DF': 5, 'VASC': 6}
        row = df[df['image'] == img_name]
        clss = row.columns[(row == 1).iloc[0]][0]
        class_ID = mapping[clss]

        bounding_boxes.append(f"{class_ID} {x_norm} {y_norm} {w_norm} {h_norm}")

    if bounding_boxes:
        output_file = f"/Skin-Cancer-Detection-App/backend/detection/data/labels/{img_name}.txt"
        
        with open(output_file, "w") as file:
            for bbox in bounding_boxes:
                file.write(f"{bbox}\n")

masks_directory = "/Skin-Cancer-Detection-App/backend/detection/data/masks"

for filename in os.listdir(masks_directory):
    file_path = os.path.join(masks_directory, filename)
    mask_to_bounding_box(file_path)


In [ ]:
import os
import cv2
import pandas as pd
import numpy as np

data_path = "/Skin-Cancer-Detection-App/backend/detection/data"
metadata_file = os.path.join(data_path, "metadata.csv")
masks_path = os.path.join(data_path, "masks")
labels_path = os.path.join(data_path, "labels")

mapping = {'MEL': 0, 'NV': 1, 'BCC': 2, 'AKIEC': 3, 'BKL': 4, 'DF': 5, 'VASC': 6}

os.makedirs(labels_path, exist_ok=True)

df = pd.read_csv(metadata_file)

def get_bounding_boxes(mask):
    boxes = []
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        boxes.append((x, y, w, h))
    return boxes

for _, row in df.iterrows():
    image_id = row["image"]
    mask_file = os.path.join(masks_path, f"{image_id}.png")
    
    if not os.path.exists(mask_file):
        continue
    
    mask = cv2.imread(mask_file, cv2.IMREAD_GRAYSCALE)
    h, w = mask.shape
    bounding_boxes = get_bounding_boxes(mask)
    
    class_id = np.argmax(row[1:].values)  
    yolo_class = mapping[list(mapping.keys())[class_id]]
    
    label_file = os.path.join(labels_path, f"{image_id}.txt")
    with open(label_file, "w") as f:
        for x, y, bw, bh in bounding_boxes:
            x_center = (x + bw / 2) / w
            y_center = (y + bh / 2) / h
            bw = bw / w
            bh = bh / h
            f.write(f"{yolo_class} {x_center} {y_center} {bw} {bh}\n")


In [ ]:
import cv2
import os

label_folder = '/Skin-Cancer-Detection-App/backend/detection/data/labels'
image_folder = '/Skin-Cancer-Detection-App/backend/detection/data/images'

def draw_bboxes(image_path, label_path):

    img = cv2.imread(image_path)
    
    with open(label_path, 'r') as f:
        for line in f:

            label_data = line.strip().split()
            class_id = int(label_data[0])
            xmin = int(float(label_data[1]) * img.shape[1])  
            ymin = int(float(label_data[2]) * img.shape[0])
            xmax = int(float(label_data[3]) * img.shape[1])
            ymax = int(float(label_data[4]) * img.shape[0])
            
            cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
            
            label_text = str(class_id)
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(img, label_text, (xmin, ymin-10), font, 0.9, (0, 255, 0), 2)

    cv2.imshow('Image with Bounding Boxes and Labels', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

count = 0
for image_name in os.listdir(image_folder):
    if image_name.endswith('.jpg') and count < 1:  
        label_name = image_name.replace('.jpg', '.txt')  
        image_path = os.path.join(image_folder, image_name)
        label_path = os.path.join(label_folder, label_name)
        
        if os.path.exists(label_path):
            draw_bboxes(image_path, label_path)
            count += 1


In [ ]:
import os
import cv2
import pandas as pd
import numpy as np

data_path = "/Skin-Cancer-Detection-App/backend/detection/data"
metadata_file = os.path.join(data_path, "metadata.csv")
masks_path = os.path.join(data_path, "masks")
labels_path = os.path.join(data_path, "labels")
images_path = os.path.join(data_path, "images")  
output_path = os.path.join(data_path, "visualized")  

mapping = {'MEL': 0, 'NV': 1, 'BCC': 2, 'AKIEC': 3, 'BKL': 4, 'DF': 5, 'VASC': 6}
class_colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255), (0, 255, 255), (128, 128, 128)]

os.makedirs(labels_path, exist_ok=True)
os.makedirs(output_path, exist_ok=True)

df = pd.read_csv(metadata_file)

def get_bounding_boxes(mask):
    boxes = []
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        boxes.append((x, y, w, h))
    return boxes

for i, (_, row) in enumerate(df.iterrows()):
    if i >= 50:
        break  
    
    image_id = row["image"]
    mask_file = os.path.join(masks_path, f"{image_id}.png")
    image_file = os.path.join(images_path, f"{image_id}.jpg")  
    
    if not os.path.exists(mask_file) or not os.path.exists(image_file):
        continue
    
    mask = cv2.imread(mask_file, cv2.IMREAD_GRAYSCALE)
    image = cv2.imread(image_file)
    h, w = mask.shape
    bounding_boxes = get_bounding_boxes(mask)
    
    class_id = np.argmax(row[1:].values)  
    yolo_class = mapping[list(mapping.keys())[class_id]]
    
    label_file = os.path.join(labels_path, f"{image_id}.txt")
    with open(label_file, "w") as f:
        for x, y, bw, bh in bounding_boxes:
            x_center = (x + bw / 2) / w
            y_center = (y + bh / 2) / h
            norm_bw = bw / w
            norm_bh = bh / h
            f.write(f"{yolo_class} {x_center} {y_center} {norm_bw} {norm_bh}\n")
            
            x2 = x + bw
            y2 = y + bh
            cv2.rectangle(image, (x, y), (x2, y2), class_colors[yolo_class], 2)
            cv2.putText(image, list(mapping.keys())[class_id], (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, class_colors[yolo_class], 2)
    
    output_file = os.path.join(output_path, f"{image_id}.jpg")
    cv2.imwrite(output_file, image)
